In [84]:
from typing import Dict
from collections import Counter, defaultdict
import json
from pprint import pprint

from py2neo import Graph, Node, Relationship

In [85]:
with open("classes.json") as f:
    classes_info = json.load(f)
    
# with open("cecs_classes_2022.json") as f:
#     classes = json.load(f)

In [86]:
graph = Graph("bolt://localhost:7687")

In [87]:
NODES = defaultdict(lambda : Node("program"))
for item in classes_info:
    node = Node("class")
    node['course_code'] = item['id']
    node['name'] = item['name']
    node['description'] = item['description']
    node['n_units'] = item['n_units']
    node['requisites_raw'] = item['requisites_raw']
    node['entities'] = item['entities']
    graph.create(node)
    
    NODES[node['course_code']] = node

In [88]:
PREREQ = Relationship.type("PREREQ") # completed
INCOMPATIBLE = Relationship.type("INCOMPATIBLE") # incompatible
DEGREE = Relationship.type("DEGREE") # studying
ENROLLED = Relationship.type("CO-REQ") # enrolled
UNKNOWN = Relationship.type("UNKNOWN") # Unknown

condition_mapper = {
    'completed': PREREQ,
    'incompatible': INCOMPATIBLE,
    'studying': DEGREE,
    'enrolled': ENROLLED,
    'Unknown': UNKNOWN
}

def write_edges(item, src_node, op='and'):
    if type(item) == list:
        for elem in item:
            write_edges(elem, src_node, op)
    elif type(item) == dict:
        if 'operator' in item and item['operator']:
            if type(item['operator']) == dict:
                for key in item['operator'].keys():
                    write_edges(item['operator'][key], src_node, key)
        if 'condition' in item and item['condition'] and item['condition'] in condition_mapper:
            edge_f = condition_mapper[item['condition']]
            if len(item['programs']) > 0:
                for name in item['programs']:
                    graph.create(edge_f(src_node, NODES[name]))
            if len(item['classes']) > 0:
                for name in item['classes']:
                    graph.create(edge_f(src_node, NODES[name]))

# write_edges(item['requisites'], NODES[item['course_code']])
for item in classes_info:
    write_edges(item['requisites'], NODES[item['id']])